In [20]:
# 1D with central differencing scheme

import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# TODO: Add reference later
"""
A property φ is transported by means of convection and diffusion through
the one-dimensional domain; the boundary conditions are φ0 = 1 at x = 0 and 
φL = 0 at x = L. Using five equally spaced cells and the central differencing 
scheme for convection and diffusion, calculate the distribution of φ as a 
function of x for (i) Case 1: u = 0.1 m/s, (ii) Case 2: u = 2.5 m/s, 
and compare the results with the analytical solution (iii) Case 3: recalculate 
the solution for u = 2.5 m/s with 20 grid nodes and compare the results with 
the analytical solution. The following data apply: length L = 1.0 m, ρ = 1.0 kg/m3
, Γ = 0.1 kg/m.s.
"""

import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve

# Parameters
L = 1.0              # Length in m
rho = 1.0            # Density in kg/m3
k = 0.1              # Diffusivity in kg/m.s
u = 0.1              # Velocity in m/s
n_nodes = 5
dx = L / n_nodes

# Boundary conditions
phi_xA = 1
phi_xB = 0

# Calculate diffusion and mass flux terms
D = k / dx            # Diffusion conductance
F = u * rho           # Mass flux per unit area

def numerical_sol(D, F, n_nodes):
    # Initialise the right-hand side vector
    b = np.zeros(n_nodes)

    # Initialise the diagonals of the matrix
    a_W = D + 0.5 * F
    a_E = D - 0.5 * F
    a_P = a_W + a_E 

    # Set up the main diagonal, super-diagonal, and sub-diagonal
    main_diag = np.full(n_nodes, a_P)
    sup_diag = np.full(n_nodes - 1, -a_E)
    sub_diag = np.full(n_nodes - 1, -a_W)

    # Modify the first and last nodes for boundary conditions
    main_diag[0] = a_E + (2 * D + F)
    b[0] = (2 * D + F) * phi_xA
    main_diag[-1] = a_W + (2 * D - F)
    b[-1] = (2 * D - F) * phi_xB

    # Create a sparse tridiagonal matrix
    diagonals = [main_diag, sup_diag, sub_diag]
    A = diags(diagonals, [0, 1, -1], format='csr')

    # Solve the linear system
    return spsolve(A, b)

def get_xs(L, n_nodes):
    dx = L/n_nodes
    return np.linspace(0, L - dx, n_nodes) + dx / 2
    


In [21]:
def analytical_sol(x):
    return (phi_xB - phi_xA) * (np.exp(rho*u*x/k) - 1)/(np.exp(rho*u*L/k) - 1) + phi_xA

phi_dist_act = analytical_sol(get_xs(L, n_nodes))
phi_dist_num = numerical_sol(D, F, n_nodes)


In [ ]:
import matplotlib.pyplot as plt

def plot(title='', show_num_line=False):
    # Calculate control volume nodes
    x_grid = get_xs(L, n_nodes)

    plt.scatter(x_grid, phi_dist_num, marker='D', color='k')
    if show_num_line:
        plt.plot(x_grid, phi_dist_num, color='k')
    plt.plot(x_grid, phi_dist_act, color='k')
    plt.legend(['Numerical solution (CD)', 'Exact solution'])
    plt.title(title)
    plt.xlabel('Distance (m)')
    plt.ylabel('φ')
    plt.show()

In [ ]:
plot(title='u=0.1m/s')

In [ ]:
# Case 2

F = 2.5
D = 0.1/0.2

phi_dist_num = numerical_sol(D, F, n_nodes)

plot(title='u=2.5m/s', show_num_line=True)

In [ ]:
# Case 3
u = 2.5
n_nodes = 20
dx = L/n_nodes
F = rho*u
D = k/dx

phi_dist_act = analytical_sol(get_xs(L, n_nodes))
phi_dist_num = numerical_sol(D, F, n_nodes)
plot()

In [ ]:
import numpy as np
# from math import max
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve


# Hybrid scheme

# Parameters
L = 1.0              # Length in m
rho = 1.0            # Density in kg/m3
k = 0.1              # Diffusivity in kg/m.s
u = 2.5              # Velocity in m/s
n_nodes = 5
dx = L / n_nodes

# Boundary conditions
phi_xA = 1
phi_xB = 0

Fe = Fw = F = rho * u
De = Dw = D = k/dx
S_P = 0


# Internal nodes
aW = max(Fw, (Dw + 0.5 * Fw), 0)
aE = max(-Fe, (De - 0.5 * Fe), 0)
aP = aW + aE + (Fe - Fw) - S_P

main_diag = np.full((n_nodes,), aP)
sub_diag = np.full((n_nodes - 1,), -aW)
sup_diag = np.full((n_nodes - 1), aE)
b = np.zeros_like(main_diag)

# Boundary nodes
S_P = -(2 * D + F)
main_diag[0] = -S_P
b[0] = (2 * D + F) * phi_xA

S_P = -2 * D
main_diag[-1] = aW - S_P
b[-1] = 2 * D * phi_xB

A = diags([main_diag, sub_diag, sup_diag], [0, -1, 1], format='csr')

x_sol = spsolve(A, b)
x_sol

In [ ]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt


def func(y, t):
    y1, y2, y3 = y
    dydt = [-0.5*y1, 0.5*y1 - 0.25*y2, 0.25*y2-(1/6)*y3]
    return dydt

y_init = np.array([1, 0.2, 0])
t = np.linspace(0, 10, 100)

sol = odeint(func, y_init, t)


plt.plot(t, sol)
plt.show()

In [29]:
import numpy as np
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve


# QUICK scheme

# Parameters
L = 1.0              # Length in m
rho = 1.0            # Density in kg/m3
k = 0.1              # Diffusivity in kg/m.s
u = 0.2              # Velocity in m/s
n_nodes = 5
dx = L / n_nodes

# Boundary conditions
phi_xA = 1
phi_xB = 0

Fe = Fw = F = rho * u
De = Dw = D = k/dx

alpha_w = 1 if Fw > 0 else 0
alpha_e = 1 if Fe > 0 else 0


aW = Dw + (6/8)*alpha_w*Fw + (1/8)*alpha_e*Fe + (3/8)*(1-alpha_w)*Fw
aWW = -(1/8)*alpha_w*Fw
aE = De - (3/8)*alpha_e*Fe - (6/8)*(1-alpha_e)*Fe - (1/8)*(1-alpha_w)*Fw
aEE = (1/8)*(1-alpha_e)*Fe

aP = aW + aE + aWW + aEE + (Fe-Fw)

main_diag = np.full((n_nodes,), aP)
sub_diag_1 = np.full((n_nodes - 1,), -aW)
sub_diag_2 = np.full((n_nodes - 2,), -aWW)
sup_diag_1 = np.full((n_nodes -1, ), -aE)
sup_diag_2 = np.full((n_nodes - 2,), -aEE)

b = np.zeros_like(main_diag)

# Node 1
D_A = k/dx
F_A = rho*u
aWW = 0
aW = 0 
aE = De + (1/3)*D_A - (3/8)*Fe
S_P = -((8/3)*D_A + (2/8)*Fe + F_A)
Su = ((8/3)*D_A + (2/8)*Fe + F_A)*phi_xA

main_diag[0] = aWW + aW + aE + (Fe-Fw) - S_P
sup_diag_1[0] = -aE
b[0] = Su


# Node 2
aWW = 0
aW = Dw + (7/8)*Fw + (1/8)*Fe 
aE = De - (3/8)*Fe 
S_P = (1/4)*Fw 
Su = -(1/4)*Fw*phi_xA

main_diag[1] = aWW + aW + aE + (Fe-Fw) - S_P
sub_diag_1[0] = -aW
sup_diag_1[1] = -aE

b[1] = Su


# Last Node
D_B = k/dx
F_B = rho*u
aWW = -(1/8)*Fw
aW = Dw + (1/3)*D_B + (6/8)*Fw
aE = 0
S_P = -((8/3)*D_B - F_B)
Su = ((8/3)*D_B - F_B)*phi_xB

main_diag[-1] = aWW + aW + aE + (Fe-Fw) - S_P
sub_diag_1[-1] = -aW
sub_diag_2[-1] = -aWW

b[-1] = Su

A = diags([main_diag, sub_diag_1, sub_diag_2 , sup_diag_1, sup_diag_2], [0, -1, -2, 1, 2], format='csr')

x_sol = spsolve(A, b)
phi_dist_act = analytical_sol(get_xs(L, n_nodes))
x_sol, phi_dist_act

[[ 2.175      -0.59166667  0.          0.          0.        ]
 [-0.7         1.075      -0.425       0.          0.        ]
 [ 0.025      -0.675       1.075      -0.425       0.        ]
 [ 0.          0.025      -0.675       1.075      -0.425     ]
 [ 0.          0.          0.025      -0.81666667  1.925     ]]


(array([0.96482596, 0.87069826, 0.73087636, 0.52256802, 0.21220362]),
 array([0.96534656, 0.8713239 , 0.73105858, 0.5218073 , 0.20964108]))

In [22]:
phi_dist_act

array([0.93879298, 0.79639032, 0.62245933, 0.41001954, 0.15054499])